In [1]:
import sys
import os
import warnings
import torch
import requests
import json
import csv
from esm import Alphabet, FastaBatchedDataset, pretrained

warnings.filterwarnings("ignore")
sys.path.append("./src/")
from CLEAN.infer import *
# pre-compute esm-1b embedding for sequences in new-392 dataset
ensure_dirs("data/esm_data")
ensure_dirs("data/pretrained")

#### input 

In [2]:
# input
# target_protein_fasta_file = './data/yeast_genome.fasta'
organism = "ecoli" # yeast ecoli human
if organism == "yeast":
    target_protein_fasta_file = './data/yeast_genome.fasta'
elif organism == "ecoli":
    target_protein_fasta_file = './data/ecoli_genome.fasta'
elif organism == "human":
    target_protein_fasta_file = './data/human_genome.fasta'
# output
file_name = target_protein_fasta_file.split('/')[-1].split('.')[0]
print(file_name)

target_protein_csv_file = './data/' + file_name + '.csv'
target_protein_result_file = './results/'+ file_name +'_maxsep.csv'
clean_result_file = './results/'+ file_name +'_clean_result.json'

ecoli_genome


###  get uniprot data

In [3]:
# taxonomy_id = '83333'
# target_uniprot_fasta_file = 'data/' + taxonomy_id + '_uniprot.fasta'
# file_name = taxonomy_id   #####change this
# target_protein_fasta_file = 'data/' + file_name + '.fasta'
# target_protein_csv_file = 'data/'+file_name+'.csv'

# target_protein_result_file = './results/'+ file_name +'_maxsep.csv'
# clean_result_file = './results/'+ file_name +'_clean_result.json'

In [4]:
# def get_uniprot_data(taxonomy_id,target_uniprot_fasta_file):
#     response = requests.get(f'https://rest.uniprot.org/uniprotkb/stream?download=true&format=fasta&query=%28%28taxonomy_id%3A{taxonomy_id}%29%29')
#     with open(target_uniprot_fasta_file, 'w') as f:
#         f.write(response.text)

In [5]:
# get_uniprot_data(taxonomy_id,target_uniprot_fasta_file)

In [6]:
# def change_uniprot_fasta(input_filename, output_filename):
#     with open(input_filename, 'r') as file:
#         fasta_content = file.read()
#     # 将文件内容按行分割
#     lines = fasta_content.split('\n')
#     result_lines = []
#     for line in lines:
#         # 如果是标识符行，提取标识符并添加到结果列表
#         if line.startswith('>'):
#             identifier = line.split('|')[1]
#             result_lines.append(f'>{identifier}')
#         else:
#             result_lines.append(line)
#     # 将结果列表合并成字符串
#     result_fasta = '\n'.join(result_lines)
#     # 将结果写入文件
#     with open(output_filename, 'w') as output_file:
#         output_file.write(result_fasta)

In [7]:
# change_uniprot_fasta(target_uniprot_fasta_file, target_protein_fasta_file)

### fasta2csv & embedding

In [8]:
def fasta_to_csv(fasta_name, csv_name):
    with open(fasta_name, 'r') as fastafile, open(csv_name, 'w') as outfile:
        csv_rows = []
        fasta_id = ''
        for line in fastafile:
            line = line.strip()
            if line.startswith('>'):
                if fasta_id:
                    csv_rows.append([fasta_id, fasta_sequence])
                fasta_id = line[1:]
                fasta_sequence = ''
            else:
                fasta_sequence += line
        if fasta_id:
            csv_rows.append([fasta_id, fasta_sequence])
        header = ['Entry', 'Sequence']
        csv_rows.insert(0, header)
        csvwriter = csv.writer(outfile, delimiter='\t')
        csvwriter.writerows(csv_rows)

In [9]:
fasta_to_csv(target_protein_fasta_file, target_protein_csv_file)

In [14]:
retrive_esm1b_embedding(file_name)

Transferred model to GPU
Read data/ecoli_genome.fasta with 4431 sequences
Processing 1 of 358 batches (113 sequences)
Processing 2 of 358 batches (73 sequences)
Processing 3 of 358 batches (62 sequences)
Processing 4 of 358 batches (56 sequences)
Processing 5 of 358 batches (51 sequences)
Processing 6 of 358 batches (48 sequences)
Processing 7 of 358 batches (45 sequences)
Processing 8 of 358 batches (44 sequences)
Processing 9 of 358 batches (41 sequences)
Processing 10 of 358 batches (39 sequences)
Processing 11 of 358 batches (38 sequences)
Processing 12 of 358 batches (37 sequences)
Processing 13 of 358 batches (36 sequences)
Processing 14 of 358 batches (35 sequences)
Processing 15 of 358 batches (33 sequences)
Processing 16 of 358 batches (33 sequences)
Processing 17 of 358 batches (32 sequences)
Processing 18 of 358 batches (31 sequences)
Processing 19 of 358 batches (30 sequences)
Processing 20 of 358 batches (29 sequences)
Processing 21 of 358 batches (29 sequences)
Processing

## predict

In [11]:
from CLEAN.infer import *
# testing on new-392 dataset with pretrained weights trained on 100% split
# infer_pvalue("split100",file_name, p_value=1e-5, nk_random=20,
#              report_metrics=True, pretrained=True)
infer_maxsep("split100",file_name, report_metrics=True, pretrained=True)

The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([4431, 128])


100%|██████████| 5242/5242 [00:00<00:00, 23833.91it/s]


Calculating eval distance map, between 4431 test ids and 5242 train EC cluster centers


4431it [00:05, 822.44it/s]


## result

In [12]:
import pandas as pd
def get_clean_predict_result(target_protein_result_file,clean_result_file):
    data = []
    max_columns = 0
    with open(target_protein_result_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            # 获取当前行的最大列数
            max_columns = max(max_columns, len(row))
            data.append(row)
    for row in data:
        row.extend([''] * (max_columns - len(row)))

    tmp_res = pd.DataFrame(data)
    for i in range(1,max_columns):
        tmp_res[i] = tmp_res[i].apply(lambda x:x.split('/')[0].split(':')[1] if '/' in x else x)
    tmp_res

    clean_result = {}
    for index,row in tmp_res.iterrows():
        ec_list = []
        for i in range(1,max_columns):
            ec_list.append(row[i])
        clean_result[row[0]] = [x for x in ec_list if x != '']
        
    with open(clean_result_file, 'w') as json_file:
        json.dump(clean_result, json_file)    

In [13]:
get_clean_predict_result(target_protein_result_file,clean_result_file)